In [3]:
!pip install transformers
!pip install -U sentence-transformers

     |████████████████████████████████| 4.0 MB 5.5 MB/s 
     |████████████████████████████████| 596 kB 40.8 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 38.8 MB/s 
     |████████████████████████████████| 895 kB 58.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 34.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=2ad16f04c49a1ff3b378a351eb8a24e04635926e8e4fdfd8d5df1df0acecd520
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers


In [1]:
!pip install rank_bm25

In [4]:
import pandas as pd
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
from transformers import DistilBertTokenizerFast
import re
import os
import spacy
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np
from rank_bm25 import BM25Okapi

In [5]:
df = pd.read_csv('context_ques_ans.csv')

In [6]:
context = list(df['Context'])
ques = list(df['Question'])
ans = list(df['Answer'])

In [ ]:
#for fine tuning Bert

ans_dict = []
idx = 0 
for a in ans : 
    temp_dict = {}
    temp_dict['text'] = a
    start_idx = context[idx].lower().find(a.lower())
    end_idx = start_idx + len(a)
    temp_dict['answer_start'] = start_idx
    temp_dict['answer_end'] = end_idx
    if start_idx > end_idx or start_idx <0 or end_idx <0 :
      print(start_idx, end_idx, context[idx])
    ans_dict.append(temp_dict)
    if context[idx].lower().find(a.lower()) == -1 :
        print(ans[idx])
    idx+=1

In [7]:
# setup GPU/CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [22]:

# initialize the tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model.to(device)
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')


Downloading:   0%|          | 0.00/443 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# tokenize
train_encodings = tokenizer(context, ques, truncation=True, padding=True)


In [ ]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        #print(answers)
        #break
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift position until found
        shift = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

# apply function to our data
add_token_positions(train_encodings, ans_dict)

In [21]:
import torch

class createDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training and validation sets
train_dataset = createDataset(train_encodings)

NameError: ignored

In [ ]:
# Fine Tuning Bert
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm


# move model over to detected device
model.to(device)
# activate training mode of model
model.train()
# initialize adam optimizer with weight decay (reduces chance of overfitting)
optim = AdamW(model.parameters(), lr=0.00001)

# initialize data loader for training data
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

for epoch in range(10):
    # set model to train mode
    model.train()
    # setup loop (we use tqdm for the progress bar)
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all the tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        # train model on batch and return outputs (incl. loss)
        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)
        # extract loss
        loss = outputs[0]
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [8]:
with open('Mahabharta_dataset.txt', 'r') as file:
    file_content_raw = file.read()

In [9]:
# splitting passage-wise
passages = file_content_raw.split('\n')

In [10]:
# returns the cleaned text
def cleaned_text(text):
    text = text.strip()
    text = re.sub(r'\n', ' ', text)
    text = text.replace(r'[^0-9a-zA-Z:,]+','')
    text = text.strip()
    return text

In [11]:
passages = [cleaned_text(passage) for passage in passages]
passages = list(filter(lambda x: len(x) > 20, passages))

In [12]:
# creating passage dictionary
passage_dict = {
    i : passage for i, passage in zip(range(len(passages)), passages)
}

In [13]:
SPACY_MODEL = os.environ.get('SPACY_MODEL', 'en_core_web_sm')

In [14]:
nlp = spacy.load(SPACY_MODEL)

In [15]:
def create_corpus(passages):
    
    corpus = []
    for i, passage in zip(range(len(passages)), passages):
        text = remove_stopwords(passage)
        tokens = [token.text.lower() for token in nlp(text) if len(token) >= 2]
        # add passage id as first token
        tokens.insert(0, str(i))
        corpus.append(tokens)
        
    return corpus

In [16]:
corpus = create_corpus(passages)

In [17]:
bm25_model = BM25Okapi(corpus=corpus) 

In [ ]:
for i in range(len(ques)) : 
  docs = bm25_model.get_top_n(query=ques[i].strip().split(), documents=corpus, n=5)
  passage_id = int(docs[0][0])
  passage = passage_dict[passage_id]
  
  print('QUES : ' , ques[i], '\nTRUE ANS :' , ans[i])
  count = 0 
  for d in docs:
    passage_id = int(d[0])
    passage = passage_dict[passage_id]
    input_ids = tokenizer.encode(cleaned_text(ques[i]), passage)
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_idx+1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    if len(input_ids) > 500 :
      continue 
    output = model(torch.tensor([input_ids]).to(device),  token_type_ids=torch.tensor([segment_ids]).to(device))
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end > answer_start:
        answer = " ".join(tokens[answer_start:answer_end+1])
        answer = tokens[answer_start]
        for k in range(answer_start+1, answer_end+1):
            if tokens[k][0:2] == "##":
                answer += tokens[k][2:]
            else:
                answer += " " + tokens[k]
        print(count,'th ANSWER :' , answer)
        count+=1
    else:
        print("I am unable to find the answer to this question. Can you please ask another question?")
    

In [19]:
def answerMyQues(ques) : 
  docs = bm25_model.get_top_n(query=ques.strip().split(), documents=corpus, n=5)
  passage_id = int(docs[0][0])
  passage = passage_dict[passage_id]
  count = 0 
  for d in docs:
    passage_id = int(d[0])
    passage = passage_dict[passage_id]
    input_ids = tokenizer.encode(cleaned_text(ques), passage)
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_idx+1
    num_seg_b = len(input_ids) - num_seg_a
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    if len(input_ids) > 500 :
      continue 
    output = model(torch.tensor([input_ids]).to(device),  token_type_ids=torch.tensor([segment_ids]).to(device))
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)
    if answer_end > answer_start:
        answer = " ".join(tokens[answer_start:answer_end+1])
        answer = tokens[answer_start]
        for k in range(answer_start+1, answer_end+1):
            if tokens[k][0:2] == "##":
                answer += tokens[k][2:]
            else:
                answer += " " + tokens[k]
        #print(count,'th ANSWER :' , answer)
        return answer
        count+=1
    else:
        print("I am unable to find the answer to this question. Can you please ask another question?")


    

In [23]:
while(True):
  print('Answer: ',answerMyQues(input("Enter the question: ")))
  print('\n')
  if input('Do you want to ask more question? (y/n): ') != 'y':
    break

Enter the question: who was pandu
I am unable to find the answer to this question. Can you please ask another question?
I am unable to find the answer to this question. Can you please ask another question?
Answer:  arjuna


Do you want to ask more question? (y/n): y
Enter the question: who was island born
Answer:  kunti


Do you want to ask more question? (y/n): n
